
--* Table1: 3-way ANOVA
* Table2: 2-way anova, alberta data
* Table S1: Genome list, accessions, habitat, clade A/B, other metadata.
* Table S3: Presence/Absence of pan-genome and targeted genes by genome
* Table S5: Pagel data

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import scikit_posthocs as sp

In [2]:
long = pd.read_table('genomes_vs_features_longform.csv', sep=',')
meta = pd.read_table('genome_labels.csv', sep=',')

def rename_clade(string):
    if type(string) == str:
        return string.replace('Clade', 'Type')
    return string

long['Type'] = long['Clade'].map(rename_clade)
loc_abbr_map = {'United Kingdom': 'UK', 
                'Canada/Alberta': 'Ab'}
long['Sampling Location'] = long.apply(lambda row: ' '.join([row['Habitat'], loc_abbr_map[row['Country/Province']]]), axis=1)

In [3]:
counts = long[['Isolate', 'Country/Province', 'Habitat', 'Type', 'source', 'Presence']]\
            .groupby(['Isolate', 'Country/Province', 'Habitat', 'Type', 'source',])\
            .sum().reset_index()

In [4]:
counts

,Isolate,Country/Province,Habitat,Type,source,Presence
0,ERR1007500,United Kingdom,Agricultural,Type A,AMR,18.0
1,ERR1007500,United Kingdom,Agricultural,Type A,Genomic Island,4.0
2,ERR1007500,United Kingdom,Agricultural,Type A,Metal,5.0
3,ERR1007500,United Kingdom,Agricultural,Type A,Novel Plasmid,0.0
4,ERR1007500,United Kingdom,Agricultural,Type A,Phage,3.0
...,...,...,...,...,...,...
8899,SRR14026555,Canada/Alberta,Clinical,Type A,Metal,6.0
8900,SRR14026555,Canada/Alberta,Clinical,Type A,Novel Plasmid,0.0
8901,SRR14026555,Canada/Alberta,Clinical,Type A,Phage,4.0
8902,SRR14026555,Canada/Alberta,Clinical,Type A,Plasmid,5.0


In [5]:
long.head()

,Isolate,Feature,Presence,Habitat,Country/Province,Clade,source,Type,Sampling Location
0,SRR14026455,AA083,0.0,Wastewater Agr.,Canada/Alberta,Clade B,Plasmid,Type B,Wastewater Agr. Ab
1,SRR14026462,AA083,0.0,Natural Water,Canada/Alberta,Clade A,Plasmid,Type A,Natural Water Ab
2,SRR14026496,AA083,0.0,Natural Water,Canada/Alberta,Clade A,Plasmid,Type A,Natural Water Ab
3,SRR14026467,AA083,0.0,Natural Water,Canada/Alberta,Clade A,Plasmid,Type A,Natural Water Ab
4,SRR14026460,AA083,0.0,Natural Water,Canada/Alberta,Clade B,Plasmid,Type B,Natural Water Ab


## Table X: Unassigned. Feature presences/percentages by sampling location

In [6]:
for feat in long['source'].unique():
    data = long[long['source']==feat]
    sort_order = ['Presence', 'Percentage']
    frames = []
    
    total = long[long['source']==feat].groupby('Feature').sum()
    total = total.join(total.div(1273).rename({'Presence': "Percentage"}, axis=1))
    total.sort_values(by='Presence', ascending=False, inplace=True)
    
    for col in ['Type', 'Sampling Location']:
        p = data.groupby([col, 'Feature']).agg({'Presence': "sum"})
        c = data.groupby([col]).agg({'Isolate': "nunique"})
        c.rename({'Isolate': "Presence"}, axis=1, inplace=True)
        perc = p.div(c, level=col).reset_index().pivot(index='Feature', columns=col, values='Presence')
        presence = p.reset_index().pivot(index='Feature', columns=col, values='Presence')

        perc.columns.name = ''
        presence.columns.name =''

        presence.rename({column: '{} Number Present'.format(column) for column in presence.columns}, axis=1, inplace=True)
        perc.rename({column: '{} Percent Present'.format(column) for column in perc.columns}, axis=1, inplace=True)

        vals = sorted(data[col].dropna().unique())
        for val in vals:
            sort_order.append('{} Number Present'.format(val))
            sort_order.append('{} Percent Present'.format(val))

        frames.append(presence.join(perc))
    total.join(frames)[sort_order].to_csv('tables/feature_presence_percentages/{}_summary.csv'.format(feat), sep=',')

In [7]:
counts.head()

,Isolate,Country/Province,Habitat,Type,source,Presence
0,ERR1007500,United Kingdom,Agricultural,Type A,AMR,18.0
1,ERR1007500,United Kingdom,Agricultural,Type A,Genomic Island,4.0
2,ERR1007500,United Kingdom,Agricultural,Type A,Metal,5.0
3,ERR1007500,United Kingdom,Agricultural,Type A,Novel Plasmid,0.0
4,ERR1007500,United Kingdom,Agricultural,Type A,Phage,3.0


## Table 1: 3-way ANOVA

In [6]:
intersect = counts[counts['Habitat'].isin(['Agricultural', 'Wastewater Mun.', 'Clinical'])]
intersect.rename({'Country/Province': 'Geography'}, axis=1, inplace=True)
intersect.head()

/home/alex/anaconda3/envs/biopython/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Isolate,Geography,Habitat,Type,source,Presence
0,ERR1007500,United Kingdom,Agricultural,Type A,AMR,18.0
1,ERR1007500,United Kingdom,Agricultural,Type A,Genomic Island,4.0
2,ERR1007500,United Kingdom,Agricultural,Type A,Metal,5.0
3,ERR1007500,United Kingdom,Agricultural,Type A,Novel Plasmid,0.0
4,ERR1007500,United Kingdom,Agricultural,Type A,Phage,3.0


In [7]:
model.summary()

NameError: name 'model' is not defined

In [8]:
t = intersect[intersect['source']==source]
factor_groups = t.groupby(['Type', 'Habitat', 'Geography'])

NameError: name 'source' is not defined

In [9]:
model = smf.ols('Presence ~ C(Type) + C(Habitat) + C(Geography) + C(Type):C(Habitat) +C(Type):C(Geography) + C(Habitat):C(Geography) + C(Type):C(Geography):C(Habitat)', data=intersect[intersect['source']==source]).fit()
d = sm.stats.anova_lm(model, typ=3)
d

NameError: name 'source' is not defined

In [41]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6,6))
for values, group in factor_groups:
    i, j = values
    plt.scatter(group['Type'], group['Presence'])
    

ValueError: too many values to unpack (expected 2)

<Figure size 432x432 with 0 Axes>

In [16]:
results = []
for source in intersect['source'].unique():
    model = smf.ols('Presence ~ C(Type) + C(Habitat) + C(Geography) + C(Type):C(Habitat) +C(Type):C(Geography) + C(Habitat):C(Geography) + C(Type):C(Geography):C(Habitat)', data=intersect[intersect['source']==source]).fit()
    d = sm.stats.anova_lm(model, typ=3)
    d = d[['PR(>F)']]
    d.rename({'PR(>F)': "{}_p".format(source)}, axis=1, inplace=True)
    results.append(d)
all_r = results[0]
for d in results[1:]:
    all_r = all_r.join(d)
all_r.dropna().iloc[1:].dropna()[['AMR_p', 'Metal_p', 'VF_p', 'Plasmid_p', 'Genomic Island_p', 'Phage_p']]

,AMR_p,Metal_p,VF_p,Plasmid_p,Genomic Island_p,Phage_p
C(Type),6.423692e-04,0.236498,8.917313e-02,5.872894e-01,9.585507e-01,4.687334e-02
C(Habitat),5.088880e-15,0.072621,1.288258e-09,1.736728e-31,5.027381e-40,1.142217e-08
C(Geography),8.342733e-01,0.000042,2.277079e-04,8.756278e-03,6.459591e-01,7.798546e-01
C(Type):C(Habitat),6.305390e-04,0.025376,1.462488e-04,1.054120e-04,5.263208e-07,7.414845e-04
C(Type):C(Geography),3.035935e-01,0.689953,7.124783e-01,3.621372e-01,4.420687e-01,6.105836e-01
C(Habitat):C(Geography),2.199025e-08,0.000162,6.576382e-01,2.504536e-04,1.763156e-13,1.163048e-03
C(Type):C(Geography):C(Habitat),1.763488e-04,0.462426,4.881143e-03,1.743480e-01,2.612433e-03,7.036217e-02


In [10]:
all_r.dropna().iloc[1:].dropna()[['AMR_p', 'Metal_p', 'VF_p', 'Plasmid_p', 'Genomic Island_p', 'Phage_p']].to_csv('tables/Table1__3Way_ANOVA.csv', sep=',')

In [17]:
all_r = all_r.dropna().iloc[1:].dropna()[['AMR_p', 'Metal_p', 'VF_p', 'Plasmid_p', 'Genomic Island_p', 'Phage_p']]

## Table 2: 2-Way ANOVA alberta data

In [19]:
counts.rename({'Country/Province': "Geography"}, axis=1, inplace=True)
results = []

for source in counts['source'].unique():
    model2 = smf.ols('Presence ~ C(Habitat) + C(Type) + C(Habitat):C(Type)', data=counts[(counts['source']==source) & (counts['Geography']=='Canada/Alberta')]).fit()
    d = sm.stats.anova_lm(model2, typ=3)
    d = d[['PR(>F)']]
    d.rename({'PR(>F)': "{}_p".format(source)}, axis=1, inplace=True)
    results.append(d)
all_r2way = results[0]
for d in results[1:]:
    all_r2way = all_r2way.join(d)
all_r2way.dropna().iloc[1:].dropna()[['AMR_p', 'Metal_p', 'VF_p', 'Plasmid_p', 'Genomic Island_p', 'Phage_p']]

,AMR_p,Metal_p,VF_p,Plasmid_p,Genomic Island_p,Phage_p
C(Habitat),2.316110e-40,0.002480,3.392543e-14,2.194568e-62,1.610631e-55,3.953562e-11
C(Type),2.337844e-07,0.120972,7.092580e-02,4.401716e-01,9.500725e-01,4.479566e-02
C(Habitat):C(Type),4.711946e-08,0.000257,2.318734e-05,1.763849e-08,1.162846e-09,1.931045e-03


In [27]:
all_r2way = all_r2way.dropna().iloc[1:].dropna()[['AMR_p', 'Metal_p', 'VF_p', 'Plasmid_p', 'Genomic Island_p', 'Phage_p']]
all_r2way.to_csv('tables/Table2__2Way_ANOVA.csv', sep=',')

## Table X (unassigned): Bonferroni Corrected t-test post-hocs
My assumed procedure:
1. Do the ANOVA
2. For cells in the ANOVA that have significant differences, we can do a t-test post-hoc.

Since the 2-way anova didn't have any insig. cells that were sig in three-way, just use the three-way.



In [44]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [47]:
factors = [['Type'],
           ['Habitat'],
           ['Geography'],
           ['Type', 'Habitat'], 
           ['Type', 'Geography'],
           ['Habitat', 'Geography'],
           ['Type', 'Geography', 'Habitat']]

def query(factor):
    return ':'.join(['C({})'.format(f) for f in factor])

tables3way = {}
#First, for 3-way
for source in ['AMR', 'Metal', 'VF', 'Plasmid', 'Genomic Island', 'Phage']:
    data = intersect[intersect['source']==source]
    for factor in factors:
        if all_r.loc[query(factor), '{}_p'.format(source)] <= 0.05:
            print("POST-HOC FOR {0}, {1}".format(source, '_'.join(factor)))
            groups = []
            for i, row in data.iterrows():
                groups.append(' X '.join(row[factor].values))
            data['groups'] = groups
            tu = sp.posthoc_tukey(data[['groups', 'Presence']], group_col='groups', val_col='Presence')
            """
            df = pd.DataFrame(tu.summary())
            header = df.iloc[0]
            header = [str(h) for h in header]
            df = df[1:]
            df.columns = header
            """
            tables3way['{0}_{1}'.format(source, '_'.join(factor))] = tu


POST-HOC FOR AMR, Type


/tmp/ipykernel_5568/173186406.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['groups'] = groups


POST-HOC FOR AMR, Habitat
POST-HOC FOR AMR, Type_Habitat
POST-HOC FOR AMR, Habitat_Geography
POST-HOC FOR AMR, Type_Geography_Habitat
POST-HOC FOR Metal, Geography
POST-HOC FOR Metal, Type_Habitat
POST-HOC FOR Metal, Habitat_Geography
POST-HOC FOR VF, Habitat
POST-HOC FOR VF, Geography
POST-HOC FOR VF, Type_Habitat
POST-HOC FOR VF, Type_Geography_Habitat
POST-HOC FOR Plasmid, Habitat
POST-HOC FOR Plasmid, Geography
POST-HOC FOR Plasmid, Type_Habitat
POST-HOC FOR Plasmid, Habitat_Geography
POST-HOC FOR Genomic Island, Habitat
POST-HOC FOR Genomic Island, Type_Habitat
POST-HOC FOR Genomic Island, Habitat_Geography
POST-HOC FOR Genomic Island, Type_Geography_Habitat
POST-HOC FOR Phage, Type
POST-HOC FOR Phage, Habitat
POST-HOC FOR Phage, Type_Habitat
POST-HOC FOR Phage, Habitat_Geography


In [23]:
for key, table in tables.items():
    table.to_csv('tables/posthoc_tests/{}_3way_post.csv'.format(key), sep=',')

In [28]:
all_r2way

,AMR_p,Metal_p,VF_p,Plasmid_p,Genomic Island_p,Phage_p
C(Habitat),2.316110e-40,0.002480,3.392543e-14,2.194568e-62,1.610631e-55,3.953562e-11
C(Type),2.337844e-07,0.120972,7.092580e-02,4.401716e-01,9.500725e-01,4.479566e-02
C(Habitat):C(Type),4.711946e-08,0.000257,2.318734e-05,1.763849e-08,1.162846e-09,1.931045e-03


In [48]:
factors = [['Type'],
           ['Habitat'],
           ['Habitat','Type'],]

tables2way = {}
#repeat for 2-way
for source in ['AMR', 'Metal', 'VF', 'Plasmid', 'Genomic Island', 'Phage']:
    data=counts[(counts['source']==source) & (counts['Geography']=='Canada/Alberta')]
    for factor in factors:
        if all_r2way.loc[query(factor), '{}_p'.format(source)] <= 0.05:
            print("POST-HOC FOR {0}, {1}".format(source, '_'.join(factor)))
            groups = []
            for i, row in data.iterrows():
                groups.append(' X '.join(row[factor].values))
            data['groups'] = groups
            tu = sp.posthoc_tukey(data[['groups', 'Presence']], group_col='groups', val_col='Presence')
            """
            df = pd.DataFrame(tu.summary())
            header = df.iloc[0]
            header = [str(h) for h in header]
            df = df[1:]
            df.columns = header
            """
            tables2way['{0}_{1}'.format(source, '_'.join(factor))] = tu


POST-HOC FOR AMR, Type
POST-HOC FOR AMR, Habitat
POST-HOC FOR AMR, Habitat_Type


/tmp/ipykernel_5568/387158977.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['groups'] = groups


POST-HOC FOR Metal, Habitat
POST-HOC FOR Metal, Habitat_Type
POST-HOC FOR VF, Habitat
POST-HOC FOR VF, Habitat_Type
POST-HOC FOR Plasmid, Habitat
POST-HOC FOR Plasmid, Habitat_Type
POST-HOC FOR Genomic Island, Habitat
POST-HOC FOR Genomic Island, Habitat_Type
POST-HOC FOR Phage, Type
POST-HOC FOR Phage, Habitat
POST-HOC FOR Phage, Habitat_Type


In [31]:
for key, table in tables.items():
    table.to_csv('tables/posthoc_tests/{}_2way_post.csv'.format(key), sep=',')

In [50]:
tables2way.keys()

dict_keys(['AMR_Type', 'AMR_Habitat', 'AMR_Habitat_Type', 'Metal_Habitat', 'Metal_Habitat_Type', 'VF_Habitat', 'VF_Habitat_Type', 'Plasmid_Habitat', 'Plasmid_Habitat_Type', 'Genomic Island_Habitat', 'Genomic Island_Habitat_Type', 'Phage_Type', 'Phage_Habitat', 'Phage_Habitat_Type'])

In [55]:
writer = pd.ExcelWriter('tables/posthoc_spreadsheet.xlsx',engine='xlsxwriter')
workbook = writer.book

significant = workbook.add_format({'bg_color': '#93c47d'})
medium = workbook.add_format({'bg_color': '#ffd966'})
bad = workbook.add_format({'bg_color': "#e06666"})

for source in ['AMR', 'Metal', 'VF', 'Plasmid', 'Genomic Island', 'Phage']:
    offset = 0
    worksheet = workbook.add_worksheet(source)
    writer.sheets[source] = worksheet
    for key in tables3way.keys():
        if key.startswith(source):
            worksheet.write_string(offset,0, '{0} {1} {2}'.format(source, key.replace('_',' X '), "3-way ANOVOA posthoc, Tukey's HSD"))
            offset += 1
            tables3way[key].to_excel(writer, sheet_name=source, startrow=offset, startcol=0)
            offset += tables3way[key].shape[0] + 4
    for key in tables2way.keys():
        if key.startswith(source):
            worksheet.write_string(offset,0, '{0} {1} {2}'.format(source, key.replace('_', ' X '), "2-way ANOVOA posthoc, Tukey's HSD"))
            offset += 1
            tables2way[key].to_excel(writer, sheet_name=source, startrow=offset, startcol=0)
            offset += tables2way[key].shape[0] + 4
    worksheet.conditional_format('A1:Z1000', {'type': 'cell', 
                                              'criteria': '<=',
                                              'value': 0.05,
                                              'format': significant})
    worksheet.conditional_format('A1:Z1000', {'type': 'cell', 
                                          'criteria': 'between',
                                          'minimum': 0.05,
                                          'maximum': 0.15,
                                          'format': medium})
    worksheet.conditional_format('A1:Z1000', {'type': 'cell',
                                              'criteria': '>',
                                              'value': 0.15,
                                              'format':bad})
writer.save()

## Table S3 Presence/absence of everything

In [60]:
def assign_gene(percent):
    if percent >= 0.99:
        return "Core"
    elif percent >= 0.95:
        return "Soft Core"
    elif percent >= 0.15:
        return "Shell"
    else:
        return "Cloud"

In [68]:
counts['source'].unique()

array(['AMR', 'Genomic Island', 'Metal', 'Novel Plasmid', 'Phage',
       'Plasmid', 'VF'], dtype=object)

In [72]:
frames =[]
for feat in ['AMR', 'Metal', 'VF', 'Plasmid', 'Genomic Island', 'Phage']:
    table = pd.read_table('tables/feature_presence_percentages/AMR_summary.csv', sep=',')[['Feature', 'Presence', 'Percentage']]
    table['source'] = feat
    frames.append(table)
type_counts = pd.concat(frames)    

In [74]:
type_counts['gene_type'] = type_counts['Percentage'].map(assign_gene)

In [76]:
type_counts.set_index('Feature').to_csv('tables/TableX__target_feature_core_assignment.csv', sep=',')

In [108]:
#get the roary gene presence absence table to count genes.
gpa = pd.read_table('gene_presence_absence_roary.csv', sep = ',', low_memory=False)
genomes = gpa.columns[14:] #14th column is start of genomes
# Convert to presence absence, remove reference and outgroup
roary = gpa.set_index('Gene')[genomes].T.notnull().astype(int).loc[meta['Isolate']]

roary.columns.name = ''
roary.index.name = 'Isolate'

#convert P/A to long form
roary_long = pd.melt(roary.reset_index(), id_vars=['Isolate'], var_name='Feature', value_name='Presence')

# count genomes
roary_t = roary_long.groupby('Feature').sum()
# percentage of genomes
roary_t = roary_t.join(roary_t.div(1273).rename({'Presence': 'Percentage'}, axis=1))
roary_t.sort_values(by='Presence', ascending=False, inplace=True)
# Annotate with core/cloud/shell
roary_t['source'] = 'Roary'
roary_t['gene_type'] = roary_t['Percentage'].map(assign_gene)

roary_t.to_csv('tables/TableX__roary_feature_core_assignment.csv', sep=',')


In [115]:
pa = pd.read_table('all_features_presence_absence.tsv', sep='\t', index_col=0)
all_pa = pa.join(roary, rsuffix='_roary')
all_pa.to_csv('tables/TableS3__all_presence_absence.csv', sep=',')

## Table S5 (?): Pagel vs labels

In [186]:
fe = pd.read_table('genomes_vs_features_longform.csv', sep=',')
loc_abbr_map = {'United Kingdom': 'UK', 
                'Canada/Alberta': 'Ab'}

fe['Sampling Location'] = fe.apply(lambda row: ' '.join([row['Habitat'], loc_abbr_map[row['Country/Province']]]), axis=1)
fe['Geography']= [loc_abbr_map[c] for c in fe['Country/Province']]
fe.head()

,Isolate,Feature,Presence,Habitat,Country/Province,Clade,source,Sampling Location,Geography
0,SRR14026455,AA083,0.0,Wastewater Agr.,Canada/Alberta,Clade B,Plasmid,Wastewater Agr. Ab,Ab
1,SRR14026462,AA083,0.0,Natural Water,Canada/Alberta,Clade A,Plasmid,Natural Water Ab,Ab
2,SRR14026496,AA083,0.0,Natural Water,Canada/Alberta,Clade A,Plasmid,Natural Water Ab,Ab
3,SRR14026467,AA083,0.0,Natural Water,Canada/Alberta,Clade A,Plasmid,Natural Water Ab,Ab
4,SRR14026460,AA083,0.0,Natural Water,Canada/Alberta,Clade B,Plasmid,Natural Water Ab,Ab


In [185]:
def rename_clade(string):
    if string == 'CladeB':
        return 'Type B'
    if type(string)==str:
        return string.replace('Clade', 'Type')
    return string
    

In [187]:
fe['Type'] = fe['Clade'].map(rename_clade)

In [189]:
# To sort properly 
recs = []
for col in ['Habitat', 'Geography', 'Type']:
    for value in fe[col].unique():
        recs.append({"Metadata Type": col,
                     "Metadata": value})
        
legend = pd.DataFrame.from_records(recs)
legend

,Metadata Type,Metadata
0,Habitat,Wastewater Agr.
1,Habitat,Natural Water
2,Habitat,Wastewater Mun.
3,Habitat,Clinical
4,Habitat,Agricultural
5,Geography,Ab
6,Geography,UK
7,Type,Type B
8,Type,Type A
9,Type,NaN


In [253]:
l1 = ['Agriculture',
 'Clinical',
 'Natural Water',
 'Wastewater Agr.',
 'Wastewater Municipal']
l2 = ['Agricultural',
 'Clinical',
 'Natural Water',
 'Wastewater Agr.',
 'Wastewater Mun.']

rename = {k:v for k, v in zip(l1,l2)}
rename['Clade A'] = 'Type A'
rename['Clade B'] = 'Type B'
rename['CladeB'] = 'Type B'
rename = {**rename, ** loc_abbr_map, "Alberta": "Ab"}
def fix_pagel(val):
    if val in rename.keys():
        return rename[val]
    return val

In [159]:
p_val = pd.read_table('pagel_vs_metadata_pvalue.csv', sep=',')
lr = pd.read_table('pagel_vs_metadata_LR.csv', sep=',')

feats = fe[['Feature', 'source']].drop_duplicates()

NameError: name 'fe' is not defined

In [172]:
df = pd.read_table('../../Indizio/data/pagel_features_p.csv', sep=',', index_col=0)
v = df.values
np.nanmin(v[np.nonzero(v)])

1.11022302462516e-16

In [255]:
p_val['Unnamed: 0'] = p_val['Unnamed: 0'].map(fix_pagel)
lr['Unnamed: 0'] = lr['Unnamed: 0'].map(fix_pagel)


In [260]:
results = {}
for feat in feats['source'].unique():
    f = feats[feats['source']==feat]
    f_p = p_val[['Unnamed: 0'] + list(f[f['Feature'].isin(p_val.columns)]['Feature'])]
    f_lr = lr[['Unnamed: 0'] + list(f[f['Feature'].isin(lr.columns)]['Feature'])]
    
    f_pg = f_p.melt(id_vars=['Unnamed: 0'])
    f_pg.rename({'Unnamed: 0': 'Metadata', 'variable': 'Feature', 'value': 'p value'}, axis=1, inplace=True)
    f_pg.set_index(['Feature', 'Metadata'], inplace=True)
    
    f_lr = lr[['Unnamed: 0'] + list(f[f['Feature'].isin(lr.columns)]['Feature'])]
    f_lrg = f_lr.melt(id_vars=['Unnamed: 0'])
    f_lrg.rename({'Unnamed: 0': 'Metadata', 'variable': 'Feature', 'value': 'LR'}, axis=1, inplace=True)
    f_lrg.set_index(['Feature', 'Metadata'], inplace=True)
    f_pg.join(f_lrg).to_csv('tables/pagel_vs_habitat/{}_vs_habitat_pagel.csv'.format(feat), sep=',')
    f_pg_lr = f_pg.join(f_lrg)
    data = fe[fe['source']==feat]
    perc_frames = []
    presence_frames = []

    total = data.groupby('Feature').sum()
    total = total.join(total.div(1273).rename({'Presence': "Percentage"}, axis=1))
    total.sort_values(by='Presence', ascending=False, inplace=True)

    for col in ['Type', 'Habitat', 'Geography']:
        p = data.groupby([col, 'Feature']).agg({'Presence': 'sum'})
        c = data.groupby([col]).agg({'Isolate': 'nunique'})
        c.rename({'Isolate': 'Presence'}, axis=1, inplace=True)
        perc = p.div(c, level=col).reset_index().pivot(index='Feature', columns=col, values='Presence')
        presence = p.reset_index().pivot(index='Feature', columns=col, values='Presence')

        perc.columns.name = ''
        presence.columns.name =''

        perc_frames.append(perc.reset_index().melt(id_vars=['Feature']).rename({'': 'Metadata', 'value': 'Proportion Present'},axis=1).set_index(['Feature', 'Metadata']))
        presence_frames.append(presence.reset_index().melt(id_vars=['Feature']).rename({'': 'Metadata', 'value': 'n Genomes Present'},axis=1).set_index(['Feature', 'Metadata']))

    final = f_pg_lr.reset_index().join(legend.set_index('Metadata'), on ='Metadata')\
        .sort_values(by=['Feature', 'Metadata Type', 'Metadata'])\
        .drop('Metadata Type',axis=1)\
        .set_index(['Feature', 'Metadata'])\
        .join(pd.concat(perc_frames)).join(pd.concat(presence_frames))
    final.to_csv('tables/pagel_vs_habitat/{}_pagel_with_counts.csv'.format(feat), sep=',')
    results[feat] = final